In [116]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
from scipy import sparse, io
import numpy as np

In [118]:
import h5sparse, h5py

In [119]:
data = io.loadmat('poli_large.mat')
A = data['Problem']['A'][0][0]

In [122]:
b = A[:,1231]

In [123]:
x = sparse.linalg.gmres(A=A, b=b.toarray())[0]

In [124]:
x = sparse.csc_matrix(x).T

In [233]:
n_test = 10
test = sparse.coo_matrix(np.arange(n_test, dtype='d').reshape(n_test,1))

# Write to HDF5

In [236]:
def write_sparse_hdf5(arr, name):
    n, m = arr.shape
    nnz = arr.nnz
    with h5py.File("{name}.h5".format(name=name), 'w') as f:
        arr_coo = arr.tocoo()
        
        m_dset = f.create_dataset('m', (1,), 'i')
        m_dset[...] = m
        n_dset = f.create_dataset('n', (1,), 'i')
        n_dset[...] = n
        nnz_dset = f.create_dataset('nnz', (1,), 'i')
        nnz_dset[...] = nnz
        
        row_dset = f.create_dataset('row', (nnz,), 'i')
        row_dset[...] = arr_coo.row + 1
        col_dset = f.create_dataset('col', (nnz,), 'i')
        col_dset[...] = arr_coo.col + 1
        data_dset = f.create_dataset('data', (nnz,), 'd')
        data_dset[...] = arr_coo.data
        

In [237]:
def print_sparse_hdf5(name):
    with h5py.File("{name}.h5".format(name=name)) as f_read:
        quants = ['m', 'n', 'nnz', 'row', 'col', 'data']
        for quant in quants:
            arr = f_read[quant].value
            print("{} = {}".format(quant, arr))
            print(arr.dtype)

In [238]:
write_sparse_hdf5(A, 'A')
write_sparse_hdf5(b, 'b')
write_sparse_hdf5(x, 'x')

In [239]:
write_sparse_hdf5(test, 'test')

In [235]:
print_sparse_hdf5('test')

m = [10]
int32
n = [1]
int32
nnz = [9]
int32
row = [ 2  3  4  5  6  7  8  9 10]
int32
col = [1 1 1 1 1 1 1 1 1]
int32
data = [ 1.  2.  3.  4.  5.  6.  7.  8.  9.]
float64


In [231]:
print_sparse_hdf5('b')

m = [15575]
int32
n = [1]
int32
nnz = [2]
int32
row = [1165 1232]
int32
col = [1 1]
int32
data = [-1.  1.]
float64
